# Prerequisites

- Host OS: Ubuntu 20.04 lts
- Using Docker Image 'mltooling/ml-workspace-gpu' (docker pull mltooling/ml-workspace-gpu)
- Single Nvidia GPU (RTX 3080)

메모

검사소견에는 숫자가 거의 날짜라서 숫자를 안남기는게 좋지 않을까?

각 컬럼별로 다른 전처리, 다른 모델이 필요할수도?


# 0. GPU check

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    device = torch.device("cpu")
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3050 Laptop GPU


# 1. Import packages

In [4]:
## Need to check if packages are compatible ##

# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 
# !pip install pyarrow==9.0.0

In [5]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [6]:
import os
import re
import math
import random
import numpy as np
import pandas as pd

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

from functools import partial

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining, ASHAScheduler
from transformers import (
    glue_tasks_num_labels,
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
    EarlyStoppingCallback
)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
num_cpus = 10
num_gpus = 1
seed = 1234

model_name = "xlm-roberta-base" # bert-base-multilingual-cased ; klue/roberta-base ; bert-base-cased ...

## Customize training strategy

task_data_dir = "test-model"
n_trials = 5

# 2. Import Data

In [6]:
data_name = "cardiovascular_sev_dataset_ensemble" ## IMDB / naver_movie_review / spam

dataset_raw = load_dataset('csv', data_files={'train': f'../data_split/{data_name}_train.csv',
                                          'test': f'../data_split/{data_name}_test.csv'})
dataset_raw

Using custom data configuration default-9578773962b99f77
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 2348
    })
    test: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 2348
    })
})

# 3. Modeling

## 3-1) BERT modeling 1 (past_history)

### 3-1-1)Data Preprocessing

In [7]:
# col_name = "past_history"
# dataset = dataset.remove_columns(['id', 'treatment_effect', 'examination']).rename_column("past_history", "text").shuffle(seed=seed)

# col_name = "treatment_effect"
# dataset = dataset.remove_columns(['id', 'examination', 'past_history']).rename_column("treatment_effect", "text").shuffle(seed=seed)

col_name = "examination"
dataset = dataset_raw.remove_columns(['id', 'treatment_effect', 'past_history']).rename_column("examination", "text").shuffle(seed=seed)
dataset

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c07c2807c3a93247.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-8e89a2f1d2346826.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2348
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2348
    })
})

In [8]:
# remove NA rows

dataset = dataset.filter(lambda row: pd.notnull(row["text"]))

## remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)
dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-40992392583d73b4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-3285fbfde1f989f7.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-8419c0474a9f43de.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-1eed3a96e69592ed.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 465
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 461
    })
})

In [9]:
# Download cache tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side = 'left') # truncation_side = 'left' option remains last 512 tokens

def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

dataset = dataset.map(tokenize_function, batched=True)
dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0a332a68557c297f.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 465
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 461
    })
})

In [10]:
# save raw for id
dataset_raw = dataset_raw.filter(lambda row: pd.notnull(row[col_name]))
dataset_raw

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-37502ae8acd6e97b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9578773962b99f77/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5564341e4855f9d9.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 465
    })
    test: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 461
    })
})

In [11]:
# train / evel / test
train_dataset = dataset["train"].select(range(0,math.floor(len(dataset["train"])*0.8)))
eval_dataset = dataset["train"].select(range(math.floor(len(dataset["train"])*0.8), len(dataset["train"])))
test_dataset = dataset["test"]

In [12]:
# Initialize ray

ray.shutdown()
ray.init(log_to_driver=False, ignore_reinit_error=True, num_cpus=num_cpus, num_gpus=num_gpus, include_dashboard=False)

2022-10-26 23:53:03,751	WARNING services.py:1884 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.45gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-10-26 23:53:03,845	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.0.1


### 3-1-2) Load PLM & Tokenizing

In [13]:
def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

weights = class_weight(train_dataset)
print(weights)

tensor([0.6838, 1.8600])


### 3-1-3) Check class weights

### 3-1-4) Modeling

In [14]:
# Download model and features

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
        )
    
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc") 
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    _n_gpu=1,
    adafactor=False,
    adam_beta1=0.01637772725627673,
    adam_beta2=0.43597204857665384,
    adam_epsilon=2.8232780081107114e-06,
    auto_find_batch_size=False,
    bf16=False,
    bf16_full_eval=False,
    data_seed=None,
    dataloader_drop_last=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    ddp_bucket_cap_mb=None,
    ddp_find_unused_parameters=None,
    ddp_timeout=1800,
    debug=[],
    deepspeed=None,
    disable_tqdm=False,
    do_eval=True,
    do_predict=False,
    do_train=True,
    eval_accumulation_steps=None,
    eval_delay=0,
    eval_steps=50,
    evaluation_strategy=steps,
    fp16=True,
    fp16_backend=auto,
    fp16_full_eval=False,
    fp16_opt_level=O1,
    fsdp=[],
    fsdp_min_num_params=0,
    fsdp_transformer_layer_cls_to_wrap=None,
    full_determinism=False,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    greater_is_better=True,
    group_by_length=False,
    half_precision_backend=cuda_amp,
    hub_model_id=None,
    hub_private_repo=False,
    hub_strategy=every_save,
    hub_token=<HUB_TOKEN>,
    ignore_data_skip=False,
    include_inputs_for_metrics=False,
    jit_mode_eval=False,
    label_names=None,
    label_smoothing_factor=0.0,
    learning_rate=2.8865301372814713e-05,
    length_column_name=length,
    load_best_model_at_end=True,
    local_rank=-1,
    log_level=-1,
    log_level_replica=-1,
    log_on_each_node=True,
    logging_dir=./logs,
    logging_first_step=False,
    logging_nan_inf_filter=True,
    logging_steps=500,
    logging_strategy=steps,
    lr_scheduler_type=linear,
    max_grad_norm=1.0,
    max_steps=-1,
    metric_for_best_model=accuracy,
    mp_parameters=,
    no_cuda=False,
    num_train_epochs=18,
    optim=adamw_hf,
    output_dir=.,
    overwrite_output_dir=False,
    past_index=-1,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=9,
    prediction_loss_only=False,
    push_to_hub=False,
    push_to_hub_model_id=None,
    push_to_hub_organization=None,
    push_to_hub_token=<PUSH_TO_HUB_TOKEN>,
    ray_scope=last,
    remove_unused_columns=True,
    report_to=[],
    resume_from_checkpoint=None,
    run_name=.,
    save_on_each_node=False,
    save_steps=500,
    save_strategy=steps,
    save_total_limit=None,
    seed=1234,
    sharded_ddp=[],
    skip_memory_metrics=True,
    tf32=None,
    torchdynamo=None,
    tpu_metrics_debug=False,
    tpu_num_cores=None,
    use_ipex=False,
    use_legacy_prediction_loop=False,
    use_mps_device=False,
    warmup_ratio=0.0016233748776006696,
    warmup_steps=0,
    weight_decay=0.21858973423865444,
    xpu_backend=None,
)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )


# Hyperparameter tuning with ray tune

tune_config = {
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "max_steps": -1
}

scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="objective", # Accuracy + F1
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
        "num_train_epochs": tune.randint(2, 20),
#         "per_device_train_batch_size": tune.choice([4, 8]),
        "weight_decay": tune.uniform(0.0, 0.3),
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "warmup_ratio": tune.uniform(0.0, 0.3),
        "adam_beta1": tune.loguniform(1e-2, 1),
        "adam_beta2": tune.loguniform(1e-3, 1),
        "adam_epsilon": tune.loguniform(1e-8, 1e-5),
    }, # correct_bias = True 어떻게 설정..?
)

reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_f1", "eval_accuracy", "objective", "eval_loss", "epoch"]
# )

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels = 2,
                                                           output_attentions = False,
                                                           output_hidden_states = False)


result = trainer.hyperparameter_search(
    hp_space = lambda _: tune_config,
    direction = "maximize",
    backend="ray",
    reuse_actors = True,
    n_trials=n_trials,
    resources_per_trial={"cpu": num_cpus, "gpu": num_gpus},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop=None,
    progress_reporter=reporter,
    local_dir="./test-results",
    name="tune_transformer_pbt",
    log_to_file=True,
)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d

== Status ==
Current time: 2022-10-26 23:53:19 (running for 00:00:02.41)
Memory usage on this node: 8.1/15.4 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 10.0/10 CPUs, 1.0/1 GPUs, 0.0/8.08 GiB heap, 0.0/4.04 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/notebook/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc             |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-----------------+-----------+-------------+----------------+--------------|
| _objective_5b557_00000 | RUNNING  | 172.17.0.2:4750 | 0.149299  | 4.27135e-05 |              4 |           17 |
| _objective_5b557_00001 | PENDING  |                 | 0.244549  | 1.63526e-05 |              4 |           13 |
| _objective_5b557_00002 | PENDING  

2022-10-26 23:53:45,905	ERROR trial_runner.py:987 -- Trial _objective_5b557_00000: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=4750, ip=172.17.0.2, repr=_objective)
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 349, in train
    result = self.step()
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 417, in step
    self._report_thread_runner_error(block=True)
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 589, in _report_thread_runner_error
    raise e
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 289, in run
    self._entrypoint()
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 362, in entrypoint
    return self._trainable_func(
  File "/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trai

Result for _objective_5b557_00000:
  date: 2022-10-26_23-53-23
  experiment_id: 3bbe9b20f0be4537860354248cdc938c
  hostname: abf27f834772
  node_ip: 172.17.0.2
  pid: 4750
  timestamp: 1666828403
  trial_id: 5b557_00000
  
== Status ==
Current time: 2022-10-26 23:53:52 (running for 00:00:35.17)
Memory usage on this node: 7.9/15.4 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 10.0/10 CPUs, 1.0/1 GPUs, 0.0/8.08 GiB heap, 0.0/4.04 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/notebook/test-results/tune_transformer_pbt
Number of trials: 5/5 (1 ERROR, 3 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc             |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-----------------+-----------+-------------+----------------+--------------|
| _objective_5b557_00001 | RUNNI

2022-10-26 23:53:57,031	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2022-10-26 23:54:01 (running for 00:00:43.54)
Memory usage on this node: 9.2/15.4 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 10.0/10 CPUs, 1.0/1 GPUs, 0.0/8.08 GiB heap, 0.0/4.04 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/notebook/test-results/tune_transformer_pbt
Number of trials: 5/5 (1 ERROR, 3 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc             |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-----------------+-----------+-------------+----------------+--------------|
| _objective_5b557_00001 | RUNNING  | 172.17.0.2:4869 | 0.244549  | 1.63526e-05 |              4 |           13 |
| _objective_5b557_00002 | PENDING  |                 | 0.016798  | 2.77065e-05 |              4 |           17 |
| _objective_5b557_00003 | 

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /opt/conda/lib/python3.8/site-packages/ray/_private/worker.py:2466 in wait                │
│                                                                                           │
│   2463 │   │                                                                              │
│   2464 │   │   timeout = timeout if timeout is not None else 10 ** 6                      │
│   2465 │   │   timeout_milliseconds = int(timeout * 1000)                                 │
│ ❱ 2466 │   │   ready_ids, remaining_ids = worker.core_worker.wait(                        │
│   2467 │   │   │   object_refs,                                                           │
│   2468 │   │   │   num_returns,                                                           │
│   2469 │   │   │   timeout_milliseconds,                                                  │
│                                                                                           │
│ /workspace/notebook/python/ray/_raylet.pyx:1414 in ray._raylet.CoreWorker.wait            │
│                                                                                           │
│ [Errno 2] No such file or directory: '/workspace/notebook/python/ray/_raylet.pyx'         │
│                                                                                           │
│ /workspace/notebook/python/ray/_raylet.pyx:173 in ray._raylet.check_status                │
│                                                                                           │
│ [Errno 2] No such file or directory: '/workspace/notebook/python/ray/_raylet.pyx'         │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441 in run_code  │
│                                                                                           │
│   3438 │   │   │   │   elif async_ :                                                      │
│   3439 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)        │
│   3440 │   │   │   │   else:                                                              │
│ ❱ 3441 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)              │
│   3442 │   │   │   finally:                                                               │
│   3443 │   │   │   │   # Reset our crash handler in place                                 │
│   3444 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-14-f4ebca5db1e7>:106 in <module>                                           │
│                                                                                           │
│ /opt/conda/lib/python3.8/site-packages/transformers/trainer.py:2381 in                    │
│ hyperparameter_search                                                                     │
│                                                                                           │
│   2378 │   │   │   HPSearchBackend.SIGOPT: run_hp_search_sigopt,                          │
│   2379 │   │   │   HPSearchBackend.WANDB: run_hp_search_wandb,                            │
│   2380 │   │   }                                                                          │
│ ❱ 2381 │   │   best_run = backend_dict[backend](self, n_trials, direction, **kwargs)      │
│   2382 │   │                                                                              │
│   2383 │   │   self.hp_search_backend = None                                              │
│   2384 │   │   return best_run                                                            │
│                                                         

In [ ]:
result

In [ ]:
for n, v in result.hyperparameters.items():
    setattr(trainer.args, n, v)

In [ ]:
trainer.args

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
pred = trainer.predict(test_dataset=test_dataset)
pred

In [ ]:
label_test = list(pred.label_ids)
pred_test = list(map(lambda x: x.index(max(x)), pred.predictions.tolist()))

In [ ]:
print(confusion_matrix(label_test, pred_test))

In [ ]:
accuracy = accuracy_score(label_test, pred_test)
f1 = f1_score(label_test, pred_test)
recall = recall_score(label_test, pred_test)
precision = precision_score(label_test, pred_test)

print(accuracy)
print(f1)
print(recall)
print(precision)

0-------0asfj lkdghz;ljdkfjdsa kewf';d\

In [7]:
# retreive the saved model 

model = AutoModelForSequenceClassification.from_pretrained('../sev_exam_1.0',
                                                           num_labels = 2
                                                          )

model.to('cuda')

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [11]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
        )
    
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc") 
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-11-8a53fec30ba5>:27 in <module>                                            │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'training_args' is not defined

In [ ]:
# 일련번호 or index랑 pred값 concat


In [ ]:
# activation function - gelu 

# data[data["examination"].isnull()] = 0

## pandas dataframe 만들고
# 아래 코드로 테스트

In [ ]:
# !pip install -q scikit-optimize

# from skopt import BayesSearchCV

# pipe = Pipeline(steps=[
#     ("preprocessor", StandardScaler()),
#     ("classifier", SVC())
# ])

# opt = BayesSearchCV(
#     pipe,
#     {
#         'classifier__C': (1e-1, 1e+1, 'log-uniform'),
#         'classifier__gamma': (1e-6, 1e+1, 'log-uniform'),
#         'classifier__degree': (1, 8),  # integer valued parameter
#         'classifier__kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
#     },
#     n_iter=8,
#     cv=3
# )

# opt.fit(x_train, y_train)

# print("val. score: %s" % opt.best_score_)
# print("test score: %s" % opt.score(x_test, y_test))

In [ ]:
# model_path = "test-model"
# trainer.model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

# Reference

https://bo-10000.tistory.com/154  
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html  
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms  
https://docs.ray.io/en/latest/tune/api_docs/search_space.html  
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html  
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/  
https://docs.ray.io/en/latest/tune/faq.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#population-based-training-tune-schedulers-populationbasedtraining  
https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search  
https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#optuna-tune-search-optuna-optunasearch  
https://kyunghyunlim.github.io/nlp/ml_ai/2021/09/22/hugging_face_5.html  

# Future Challenges
2. 훈련 셋이 늘어나면서 성능이 어떻게 좋아지는지, hp조합에 따라 어떻게 좋아지는지 시각화